In [1]:
from RNN_QSR import *

cuda:0


In [2]:
class PE2D(nn.Module):
    def __init__(self, d_model, Lx,Ly,device,n_encode=None):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(Lx*Ly, d_model)
        
        if type(n_encode)==type(None):
            n_encode=3*d_model//4
        for pos in range(Lx*Ly):
            x=pos//Ly
            y=pos%Ly
            # Only going to fill 3/4 of the matrix so the
            # occupation values are preserved
            for i in range(0, n_encode, 4):
                
                #x direction encoding
                pe[pos, i] = \
                math.sin(x / (10000 ** ((2 * i)/n_encode)))
                pe[pos, i + 1] = \
                math.cos(x / (10000 ** ((2 * (i + 1))/n_encode)))
                #y direction encoding
                pe[pos, i+2] = \
                math.sin(y / (10000 ** ((2 * i)/n_encode)))
                pe[pos, i + 3] = \
                math.cos(y / (10000 ** ((2 * (i + 1))/n_encode)))
                
        self.pe = pe.unsqueeze(0).to(device)
        self.L=Lx*Ly
    
    def forward(self, x):
        return x + self.pe[:,:self.L,:]

In [3]:
class SlowTransformer(Sampler):
    def __init__(self,Lx,Ly,device=device,Nh=128,decoder=False,dropout=0.0,num_layers=3, **kwargs):
        super(SlowTransformer, self).__init__(device=device)
        
        self.pe = PE2D(Nh, Lx,Ly,device)
        
        if decoder:
            #Decoder only transformer
            self.decoder_layer = nn.TransformerDecoderLayer(d_model=Nh, nhead=8, dropout=dropout)
            self.transformer = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        else:
            #Encoder only transformer
            self.encoder_layer = nn.TransformerEncoderLayer(d_model=Nh, nhead=8, dropout=dropout)
            self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        
        self.lin = nn.Sequential(
                nn.Linear(Nh,1),
                #nn.ReLU(),
                #nn.Linear(128,1),
                nn.Sigmoid()
            )
        
        
        self.set_mask(Lx*Ly)
        self.to(device)
        
    def set_mask(self, L):
        # take the log of a lower triangular matrix
        self.L=L
        self.mask = torch.log(torch.tril(torch.ones(L,L))).to(device)
        self.pe.L=L

    def forward(self, input):
        # input is shape [B,L,1]
        # add positional encoding to get shape [B,L,Nh]
        if input.shape[1]!=self.L:
            self.set_mask(input.shape[1])
        
        input=self.pe(input).transpose(1,0)
        output = self.transformer(input,self.mask)
        output = self.lin(output.transpose(1,0))
        return output
    
    
    def logprobability(self,input):
        """Compute the logscale probability of a given state
            Inputs:
                input - [B,L,1] matrix of zeros and ones for ground/excited states
            Returns:
                logp - [B] size vector of logscale probability labels
        """
        
        #Input should have shape [B,L,1]
        B,L,one=input.shape
        
        #first prediction is with the zero input vector
        data=torch.zeros([B,L,one],device=self.device)
        #data is the input vector shifted one to the right, with the very first entry set to zero instead of using pbc
        data[:,1:,:]=input[:,:-1,:]
        
        #real is going to be a set of actual values
        real=input
        #and pred is going to be a set of probabilities
        #if real[i]=1 than you multiply your conditional probability by pred[i]
        #if real[i]=0 than you multiply by 1-pred[i]
        
        #probability predictions may be done WITH gradients
        #with torch.no_grad():
        
        pred = self.forward(data)
        ones = real*pred
        zeros=(1-real)*(1-pred)
        total = ones+zeros
        #this is the sum you see in the cell above
        #add 1e-10 to the prediction to avoid nans when total=0
        logp=torch.sum(torch.log(total+1e-10),dim=1).squeeze(1)
        return logp
    def sample(self,B,L):
        """ Generates a set states
        Inputs:
            B (int)            - The number of states to generate in parallel
            L (int)            - The length of generated vectors
        Returns:
            samples - [B,L,1] matrix of zeros and ones for ground/excited states
        """
        
        #Sample set will have shape [B,L,1]
        #need one extra zero batch at the start for first pred hence input is [N,L+1,1] 
        input = torch.zeros([B,L+1,1],device=self.device)
        
        self.set_mask(L)
        #sampling can be done without gradients
        with torch.no_grad():
          for idx in range(1,L+1):
            #run the rnn on shape [B,1,1]   
            #encode the input to the proper shape
            encoded_input = input[:,:idx,:]+self.pe.pe[:,:idx,:]
                        
            #Get transformer output
            output = self.transformer(encoded_input.transpose(1,0),self.mask[:idx,:idx])
            #if probs[i]=1 then there should be a 100% chance that sample[i]=1
            #if probs[i]=0 then there should be a 0% chance that sample[i]=1
            #stands that we generate a random uniform u and take int(u<probs) as our sample
            probs=self.lin(output.transpose(1,0)[:,-1,:])
            sample = (torch.rand([B,1],device=device)<probs).to(torch.float32)
            input[:,idx,:]=sample
        #input's first entry is zero to get a predction for the first atom
        return input[:,1:,:]

In [4]:
trainsformer = SlowTransformer(4,4,Nh=32,num_layers=2)
sampleformer= SlowTransformer(4,4,Nh=32,num_layers=2)
beta1=0.9;beta2=0.999
optimizer = torch.optim.Adam(
trainsformer.parameters(), 
lr=1e-3, 
betas=(beta1,beta2)
)

In [ ]:
op=Opt()
#op.apply(sys.argv[1:])
#op.B=op.K*op.Q
#op.steps=4000
op.dir="TF"
print(op)

if op.USEQUEUE:
    queue_train(op,(trainsformer,sampleformer,optimizer))
else:
    reg_train(op)

L                             			16
Q                             			32
K                             			16
B                             			512
TOL                           			0.15
M                             			0.96875
USEQUEUE                      			True
NLOOPS                        			1
hamiltonian                   			Rydberg
steps                         			12000
dir                           			TF

Output folder path established
-0.45776822 16
2.000339984893799
0,1.98|21,-0.39|42,-0.40|63,-0.29|84,-0.40|105,-0.43|126,-0.44|147,-0.45|168,-0.46|189,-0.46|211,-0.46|232,-0.46|253,-0.46|

In [ ]:
torch.arange(4*4).reshape([4,4,1]) + torch.ones([1,4,4])

In [ ]:
pe = PE2D(40,6,6,torch.device('cpu')).pe
print(pe.shape,pe.max())

In [ ]:
plt.imshow(pe[0])

In [ ]:
torch.zeros([1,2,3]).transpose(1,0)